In [1]:
import numpy as np
from copy import deepcopy
from graphs_utils import *

In [3]:
graph1 = {
    'a': ['b','c','d'],
    'b': ['a','c'],
    'c': ['a','b','d','e'],
    'd': ['a','c','e','f'],
    'e': ['c','d','f'],
    'f': ['d','e']
}
print_graph(graph1)

a :  b  c  d
b :  a  c
c :  a  b  d  e
d :  a  c  e  f
e :  c  d  f
f :  d  e


In [4]:
# graph1.pop('b')
# print_graph(graph1)

In [5]:
# graph1['c'].remove('b')
# print_graph(graph1)

## Maximum cardinality search algorithm

In [6]:
def usun_wierzcholek(graph, key):
  del graph[key]
  for v in graph:
    for i in graph[v]:
      if i == key:
        graph[v].remove(i)
        break
  return graph

In [7]:
# maximum cardinality search algorithm
def mcs(graph):
  graph1 = deepcopy(graph)
  PEO = []
  w = [list(graph.keys()), []]
  max = 1
  for v in graph1:
    w[1].append(0)
  for i in range(len(w[0])):
    x = np.argmax(w[1])
    y = w[0][x]
    if w[1][x] > max:
      max = w[1][x]
    PEO.append(y)
    del w[0][x]
    del w[1][x]
    for j in range(len(w[0])):
      if w[0][j] in graph1[y]:
        w[1][j] += 1
    graph1 = usun_wierzcholek(graph1, y)
    graph = graph1.copy()
  return  PEO, (max + 1)

## Kolorowanie

In [8]:
def koloruj_ciec(graph):
  a, b = mcs(graph)
  for v in graph:
    ind = a.index(v)
    poprz = a[0:ind]
    kolory = list(range(1, b + 1))
    for u in poprz:
      if u in graph[v] and list(graph[u])[-1] in kolory:
        kolory.remove(list(graph[u])[-1])
    graph[v].append(kolory[0])
  return graph

In [9]:
#print_graph(graph1)
# print_graph(koloruj_ciec(graph1))

In [10]:
graph2 = {
    'a': ['b','e','f','h','k','j','i'],
    'b': ['a','c','d','e', 'k'],
    'c': ['d','b'],
    'd': ['b','c'],
    'e': ['b','a','f'],
    'f': ['a','g','h','e'],
    'g': ['f'],
    'h': ['a','f'],
    'i': ['l','j','a'],
    'j': ['l','i','k','a'],
    'k': ['j','a','b'],
    'l': ['i','j']
}
print_graph(graph2)

a :  b  e  f  h  k  j  i
b :  a  c  d  e  k
c :  d  b
d :  b  c
e :  b  a  f
f :  a  g  h  e
g :  f
h :  a  f
i :  l  j  a
j :  l  i  k  a
k :  j  a  b
l :  i  j


In [11]:
# print_graph(koloruj_ciec(graph2))

In [12]:
def BFS(graph, v):
  kolejnosc = []
  kolejka = []
  kolejka.append(v)
  while len(kolejka) > 0:
    u = kolejka.pop(0)
    kolejnosc.append(u)
    for w in graph[u]:
      if not w in kolejnosc and not w in kolejka:
        kolejka.append(w)
  return kolejnosc

In [13]:
print(BFS(graph2, 'a'))

['a', 'b', 'e', 'f', 'h', 'k', 'j', 'i', 'c', 'd', 'g', 'l']


In [14]:
def usun_wierzcholek(graph, key):
  del graph[key]
  for v in graph:
    for i in graph[v]:
      if i == key:
        graph[v].remove(i)
        break
  return graph


def pomoc(A, B):
  wynik = []
  for el in A:
    if el in B:
      wynik.append(el)
      break
  if wynik:
    return (wynik[0])


def r_list(A, B):
  C = A.copy()
  for el in B:
    if el in A:
      C.remove(el)
  return C


def p_list(A, B):
  C = []
  if not A or not B:
    return C
  else:
    for a in A:
      if a in B:
        C.append(a)
    return C


def nie_sasiad(graph, v):
  V = list(graph.keys())
  A = r_list(V, graph[v])
  A.remove(v)
  return A
  

In [15]:
A = [1,2,3,4,5,6]
B = [1,3,5,7]

print(r_list(A, B))
print(p_list(A, B))
print(nie_sasiad(graph2, 'i'))

[2, 4, 6]
[1, 3, 5]
['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k']


In [16]:
def LexBFS(graph, v):
  '''
  Metoda zwraca doskonały porządek dla grafu cieciwowego. v jest wierzcholkiem od ktorego zaczynamy szukanie doskonalego porzadku.
  '''
  U = []
  U.append(v)
  Sigma = []
  N = graph[v]
  Sigma.append(N)  # dodaje sasiadow v
  V = list(graph.keys())
  V.remove(v)
  Sigma.append(r_list(V, N))
  # print(Sigma)
  while Sigma:
    u = pomoc(V, Sigma[0])
    U.append(u)
    V.remove(u)
    # list_pomoc = Sigma.copy()
    # list_pomoc.reverse()
    pomoc_lista = Sigma.copy()
    for S in pomoc_lista:
      A = p_list(S, graph[u])
      B = p_list(S, nie_sasiad(graph, u))
      # print(S)
      # print(graph[u])
      # print(nie_sasiad(graph,u))
      # print(A,B)
      if A:
        Sigma.append(A)
      if B:
        Sigma.append(B)
      Sigma.remove(S)
    # print(Sigma)
  U.reverse()
  return U

In [17]:
lista = LexBFS(graph2, 'g')
print(lista)

['l', 'd', 'c', 'i', 'j', 'k', 'b', 'h', 'e', 'a', 'f', 'g']


In [18]:
def greedy_coloring(graph, lista):
  '''
  lista - doskonaly porzadek. Odwrocona lista to kolejnosc zachlannego kolorowania
  '''
  graph1 = deepcopy(graph)
  color_list = list(range(1, len(list(graph1.keys())))) # maksymalna liczba kolorow do pokolorowania k-wierzcholkowego grafu
  lista.reverse()
  kolejnosc = lista
  for v in kolejnosc:
    wykorzystane = []
    poprzednicy = kolejnosc[0:(kolejnosc.index(v))]
    pop_sadz = p_list(poprzednicy, graph1[v])
    for u in pop_sadz:
      color = graph1[u][-1]
      if not color in wykorzystane:
        wykorzystane.append(color)
    dostepne = r_list(color_list, wykorzystane)
    kolor = dostepne[0]
    graph1[v].append(kolor)
  return graph1


In [19]:
print_graph(greedy_coloring(graph2, lista))

a :  b  e  f  h  k  j  i  1
b :  a  c  d  e  k  2
c :  d  b  1
d :  b  c  3
e :  b  a  f  3
f :  a  g  h  e  2
g :  f  1
h :  a  f  3
i :  l  j  a  3
j :  l  i  k  a  2
k :  j  a  b  3
l :  i  j  1


In [20]:
graph3 = {'a': ['b','c','d'],
          'b': ['a','c'],
          'c': ['a','b','d','e','f'],
          'd': ['a', 'c', 'e', 'f'],
          'e': ['c','d'],
          'f': ['c','d','g'],
          'g': ['f']}

In [21]:
print_graph(graph3)

a :  b  c  d
b :  a  c
c :  a  b  d  e  f
d :  a  c  e  f
e :  c  d
f :  c  d  g
g :  f


In [22]:
lista = LexBFS(graph3, 'a')
print(lista)
print_graph(greedy_coloring(graph3, lista))

['g', 'f', 'e', 'd', 'c', 'b', 'a']
a :  b  c  d  1
b :  a  c  2
c :  a  b  d  e  f  3
d :  a  c  e  f  2
e :  c  d  1
f :  c  d  g  1
g :  f  2


In [23]:
# def koloruj_ciec2(graph, a):
#   b = mcs(graph)[1]
#   a = LexBFS(graph, a)
#   for v in graph:
#     ind = a.index(v)
#     poprz = a[0:ind]
#     kolory = list(range(1, b + 1))
#     for u in poprz:
#       if u in graph[v] and list(graph[u])[-1] in kolory:
#         kolory.remove(list(graph[u])[-1])
#     graph[v].append(kolory[0])
#   return graph

In [24]:
print_graph(koloruj_ciec2(graph2, 'g'))

NameError: ignored

#Grafy

In [ ]:
graph1 = {
    'a': ['b','c','d'],
    'b': ['a','c'],
    'c': ['a','b','d','e'],
    'd': ['a','c','e','f'],
    'e': ['c','d','f'],
    'f': ['d','e']
          }
graph2 = {
    'a': ['b','e','f','h','k','j','i'],
    'b': ['a','c','d','e', 'k'],
    'c': ['d','b'],
    'd': ['b','c'],
    'e': ['b','a','f'],
    'f': ['a','g','h','e'],
    'g': ['f'],
    'h': ['a','f'],
    'i': ['l','j','a'],
    'j': ['l','i','k','a'],
    'k': ['j','a','b'],
    'l': ['i','j']
          }
graph3 = {'a': ['b','c','d'],
          'b': ['a','c'],
          'c': ['a','b','d','e','f'],
          'd': ['a', 'c', 'e', 'f'],
          'e': ['c','d'],
          'f': ['c','d','g'],
          'g': ['f']
          }

# Pomocnicze funkcje

In [ ]:
def print_graph(graph):
  """
  Wypisuje graf zadany jako słownik pythona
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")


def usun_wierzcholek(graph, key):
  del graph[key]
  for v in graph:
    for i in graph[v]:
      if i == key:
        graph[v].remove(i)
        break
  return graph


def pomoc(A, B):
  '''
  Szuka pierwszego wierzcholka z listy A w liscie wierzcholkow B.
  '''
  wynik = []
  for el in A:
    if el in B:
      wynik.append(el)
      break
  if wynik:
    return (wynik[0])


def r_list(A, B):
  '''
  roznica list
  '''
  C = A.copy()
  for el in B:
    if el in A:
      C.remove(el)
  return C


def p_list(A, B):
  '''
  czesc wspolna list
  '''
  C = []
  if not A or not B:
    return C
  else:
    for a in A:
      if a in B:
        C.append(a)
    return C


def nie_sasiad(graph, v):
  '''
  zwraca liste wierzcholkow grafu graph ktore nie sa sasiadami wierzcholka v
  '''
  V = list(graph.keys())
  A = r_list(V, graph[v])
  A.remove(v)
  return A

# Główne algorytmy

In [ ]:
# maximum cardinality search algorithm
def mcs(graph):
  '''
  Zwraca liste kolorowania wierzcholkow oraz minimalna liczbe wymagana do pokolorowania grafu
  '''
  graph1 = deepcopy(graph)
  PEO = []
  w = [list(graph.keys()), []]
  max = 1
  for v in graph1:
    w[1].append(0)
  for i in range(len(w[0])):
    x = np.argmax(w[1])
    y = w[0][x]
    if w[1][x] > max:
      max = w[1][x]
    PEO.append(y)
    del w[0][x]
    del w[1][x]
    for j in range(len(w[0])):
      if w[0][j] in graph1[y]:
        w[1][j] += 1
    graph1 = usun_wierzcholek(graph1, y)
    graph = graph1.copy()
  return  PEO, (max + 1)


def LexBFS(graph, v):
  '''
  Metoda zwraca doskonały porządek dla grafu cieciwowego. v jest wierzcholkiem od ktorego zaczynamy szukanie doskonalego porzadku.
  '''
  U = []
  U.append(v)
  Sigma = []
  N = graph[v]
  Sigma.append(N)  # dodaje sasiadow v
  V = list(graph.keys())
  V.remove(v)
  Sigma.append(r_list(V, N))
  # print(Sigma)
  while Sigma:
    u = pomoc(V, Sigma[0])
    U.append(u)
    V.remove(u)
    # list_pomoc = Sigma.copy()
    # list_pomoc.reverse()
    pomoc_lista = Sigma.copy()
    for S in pomoc_lista:
      A = p_list(S, graph[u])
      B = p_list(S, nie_sasiad(graph, u))
      # print(S)
      # print(graph[u])
      # print(nie_sasiad(graph,u))
      # print(A,B)
      if A:
        Sigma.append(A)
      if B:
        Sigma.append(B)
      Sigma.remove(S)
    # print(Sigma)
  U.reverse()
  return U


def greedy_coloring(graph, lista):
  '''
  lista - doskonaly porzadek. Odwrocona lista to kolejnosc zachlannego kolorowania
  '''
  graph1 = deepcopy(graph)
  color_list = list(range(1, len(list(graph1.keys())))) # maksymalna liczba kolorow do pokolorowania k-wierzcholkowego grafu
  lista.reverse()
  kolejnosc = lista
  for v in kolejnosc:
    wykorzystane = []
    poprzednicy = kolejnosc[0:(kolejnosc.index(v))]
    pop_sadz = p_list(poprzednicy, graph1[v])
    for u in pop_sadz:
      color = graph1[u][-1]
      if not color in wykorzystane:
        wykorzystane.append(color)
    dostepne = r_list(color_list, wykorzystane)
    kolor = dostepne[0]
    graph1[v].append(kolor)
  return graph1

#Wykorzystanie

In [ ]:
lista = LexBFS(graph1, 'a')
print(lista)
print_graph(greedy_coloring(graph1, lista))

lista = LexBFS(graph2, 'a')
print(lista)
print_graph(greedy_coloring(graph2, lista))

lista = LexBFS(graph3, 'a')
print(lista)
print_graph(greedy_coloring(graph3, lista))